## Машинное обучение, ВМК МГУ

## Практическое задание 2

### Общая информация
Дата выдачи: 9 октября 2019

Максимальная оценка: 10 баллов + 1 бонусный балл

Мягкий дедлайн: 23:59MSK 23 октября (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 30 октября.

### О задании

В этом задании вы:
- Познакомитесь с методом решения задачи регрессии на основе метода ближайших соседей.
- Реализуете алгоритм kNN для задачи регрессии.
- Изучите методы работы с категориальными и текстовыми переменными.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.


### Формат сдачи
Для сдачи задания переименуйте получившийся файл *.ipynb в соответствии со следующим форматом: homework-practice-02-Username.ipynb, где Username — ваша фамилия и имя на латинице именно в таком порядке (например, homework-practice-02-ivanov.ipynb).

Далее отправьте этот файл на anytask в соответсвующий раздел.

In [1]:
import numpy as np
import pandas as pd

Все эксперименты в этой лабораторной работе предлагается проводить на данных соревнования New York City Airbnb Open Data: https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data#AB_NYC_2019.csv

В данной задаче предлагается предсказать цену на съем квартиры в зависимости от её параметров.

In [2]:
data = pd.read_csv('AB_NYC_2019.csv')
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [3]:
data.shape

(48895, 16)

In [4]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

id 48895
name 47906
host_id 37457
host_name 11453
neighbourhood_group 5
neighbourhood 221
latitude 19048
longitude 14718
room_type 3
price 674
minimum_nights 109
number_of_reviews 394
last_review 1765
reviews_per_month 938
calculated_host_listings_count 47
availability_365 366


In [5]:
data.isna().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

Как видите, в данных есть пропуски. Не забудьте обработать их

In [6]:
# Ваш код здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆
data['name'].fillna('', inplace=True)
data['host_name'].fillna('', inplace=True)
data['last_review'].fillna('', inplace=True)
data['reviews_per_month'].fillna(0, inplace=True)
data.isna().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

Разобъем данные на обучение и контроль.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

## Часть 1: Алгоритм kNN в задаче регрессии

<b> 1.1 (1.5 балла) </b>
Реализуйте класс `KNNRegressor`, который используя метод k ближайших соседей решает задачу регрессии. Для решение данной задачи, необходимо найти $N_k$ - k соседей, и после использовать значения их целевых переменных для предсказания:
\begin{align}
y = \frac{1}{k}\sum_{n \in N_k}w_n y_n,
\end{align}

где $w_n$ - вес каждого соседа. 

При этом `KNNRegressor` может работать в 2 режимах:
 - $uniform$ - ближайшие соседи учитываются с одинаковыми весами.
 - $distance$ - вес ближайших соседей зависит от расстояния
 
Сигнатуру методов при желании можно менять.

In [40]:
from typing import Union, Callable, Iterable, Optional
from sklearn.neighbors import NearestNeighbors


class KNNRegressor:
    def __init__(self, n_neighbors: int, metric: Union[str, Callable], mode: str = 'uniform'):
        """
        params:
            n_neighbors: number of neighbors
            metric: metric to use for distance computation
            mode: 'uniform' or 'distance'
            'uniform' - all points in each neighborhood are weighted equally
            'distance' - weight points by the inverse of their distance
        """
        self.__nn = NearestNeighbors(n_neighbors=n_neighbors, metric=metric, n_jobs=-1)
        self.__mode = mode
        self.__n_neighbors = n_neighbors
        self.metric = metric
        self.overlap_metrics = ['overlap', 'flattened_overlap', 'log_overlap']

    def fit(self, X: np.array, y: np.array) -> None:
        """
            X: data
            y: labels
        
        self.X = X
        self.y = y"""
        if not ("overlap" in str(self.metric)):
            self.__nn.fit(X, y)
        else:
            self.X = X
            
        self.y = y
            
    def find_n(self, x):
        
        
        if "overlap" in str(self.metric):
            ans_matrix = np.zeros(x.shape[0] * self.X.shape[0]).reshape(x.shape[0], self.X.shape[0])
            for q in range(self.X.shape[1]):
                ans_matrix += self.metric(x[:,q], self.X[:,q])
        else:
            ans_matrix = self.__nn.kneighbors(x)
        
        index = np.argsort(ans_matrix, axis=1)[:,:self.__n_neighbors]
        distance = np.sort(ans_matrix, axis=1)[:, :self.__n_neighbors]
        return (distance, index)


    def predict(self, X: np.array, n_neighbors: Optional[int] = None) -> np.array:
        """
            X: data
            n_neighbors: number of neighbors
        """
        distance, index = self.find_n(X)
        weight = 1 / (distance + 0.00001) if self.__mode == 'distance' else np.ones_like(distance)
        ans = np.zeros(X.shape[0])
        for i, elem in enumerate(index):

            ans[i] = np.sum(self.y[elem].T * weight[i]) / np.sum(weight[i])
        return ans
            

## Часть 2: Категориальные признаки

<b>2.1 (1 балл)</b>
Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на [третьем семинаре](https://github.com/mmp-mmro-team/mmp_mmro_fall_2019/blob/master/lecture-notes/Sem03_knn.pdf). Не забудьте, что KNNRegressor должен уметь работать с этими функциями расстояния. Как вариант, можно реализовать метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [9]:
# данные функции принимают на вход два вектора, возвращают расстояния между ними (число). 
# в  flattened и log используетсявспомогательная структура, в которой хранится информация о том, сколько раз тот или иной признак
# принимал некоторое значение на обучающей выборке

def overlap(x, z):
    answer = 0
    for i in range(len(x)):
        answer += (x[i] != z[i]).astype(int)
    return answer

 

def flattened_overlap(x, z):
    answer = 0
    for i in range(len(x)):
        if x[i] == z[i]:
            for q in matr_dist[i]:
                
                if matr_dist[i][q] / len(X_train) <= matr_dist[i][x[i]] / len(X_train):

                    answer += matr_dist[i][q] * (matr_dist[i][q] - 1) / (len(X_train) * (len(X_train) - 1))
            
            """
            for j in np.unique(x):
                if len(np.where(x == j)[0]) / len(x) <= len(np.where(x == x[i])[0]) / len(x):
                    answer += len(np.where(x == j)[0]) * (len(np.where(x == j)[0]) - 1) / (len(x) * (len(x) - 1))"""
        else:
            answer += 1
    return answer
        

def log_overlap(x, z):
    answer = 0
    for i in range(len(x)):
        if x[i] != z[i]:
            answer += np.log(matr_dist[i][x[i]] + 1) * np.log(matr_dist[i][z[i]] + 1)
    return answer

In [10]:
# эти метрики используются для получения матрицы попарных расстояний в моей функции поиска соседей, так как
# поиск соседей через NearestNeighbors с пользовательскими метриками работает слишком долго, особенно с log_overlap - около 8 часов,
# причем так и не доработал
# overlap - около 30 минут, flattened_overlap - около 4 часов. Возможно, в этом есть вина ОС, но тем не менее

# функции принимают на вход два вектора (по факту, множество объектов со значениеми какого-то одного признака), возвращают матрицу.

def overlap(x, z):
    answer = np.zeros(len(x) * len(z)).reshape(len(x), len(z))
    for i, elem in enumerate(x):
        answer[i] = (elem != z).astype(int)
    return answer

def flattened_overlap(x, z):
    answer = np.zeros(len(x) * len(z)).reshape(len(x), len(z))
    
    # т.к. смотрим на верктор из всех значений какого-то признака, название которого мы не знаем, не можем использовать структуру,
    # созданную для предыдущих функций, возвращающих число => будем находить частоты встречаемости значений прямо здесь
    
    values, counts = np.unique(x, return_counts=True)
    freq = counts / len(x)
    
    for i, elem in enumerate(x):
        
        index = np.where(elem == values)[0][0]
        q = np.where(freq <= freq[index])[0]
        unique_sum = np.sum(counts[q] * (counts[q] - 1) / (len(x) * (len(x) - 1)))
        answer[i] = (elem != z).astype(int) + (elem == z).astype(int) * unique_sum
        
    return answer

def log_overlap(x, z):
    answer = np.zeros(len(x) * len(z)).reshape(len(x), len(z))
    
    freq_x = np.zeros_like(x)
    freq_z = np.zeros_like(z)
    
    for i in range(len(x)):
        freq_x[i] = len(np.where(x[i] == x)[0])
        
    
    for i in range(len(z)):
        freq_z[i] = len(np.where(z[i] == z)[0])
    
    log_x = np.log(freq_x + 1)
    log_z = np.log(freq_z + 1)
    
    for i, elem in enumerate(x):
        
        answer[i] = (elem != z).astype(int) * log_x[i] * log_z
    return answer

In [11]:
x = np.array([1, 0, 1])
z = np.array([1, 1, 2])
log_overlap(x, z)

array([[0.        , 0.        , 0.76150001],
       [0.76150001, 0.76150001, 0.48045301],
       [0.        , 0.        , 0.76150001]])

<b>2.2 (1 балл)</b> Найдите все категориальные признаки в данных. Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Качество измеряйте с помощью RMSE.

Какая функция расстояния оказалась лучшей? Почему?

Категориальные признаки в данном случае - neighbourhood_group, neighbourhood и room_type.

In [12]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [13]:
# составляем train_set и test_set из категориальных признаков

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

cat_X_train = pd.DataFrame()

cat_X_train['neighbourhood_group'] = le.fit_transform(X_train['neighbourhood_group'])
cat_X_train['neighbourhood'] = le.fit_transform(X_train['neighbourhood'])
cat_X_train['room_type'] = le.fit_transform(X_train['room_type'])

cat_X_test = pd.DataFrame()

cat_X_test['neighbourhood_group'] = le.fit_transform(X_test['neighbourhood_group'])
cat_X_test['neighbourhood'] = le.fit_transform(X_test['neighbourhood'])
cat_X_test['room_type'] = le.fit_transform(X_test['room_type'])


In [14]:
# вспомогательная структура данных для метрик overlap (для варианта, который возвращает число, а не матрицу)
dict_ng = {}
for x in np.unique(cat_X_train['neighbourhood_group']):
    dict_ng[x] = len(np.where(x == cat_X_train['neighbourhood_group'])[0])
dict_n = {}
for x in np.unique(cat_X_train['neighbourhood']):
    dict_n[x] = len(np.where(x == cat_X_train['neighbourhood'])[0])
dict_rt = {}
for x in np.unique(cat_X_train['room_type']):
    dict_rt[x] = len(np.where(x == cat_X_train['room_type'])[0])
    
matr_dist = np.array([dict_ng, dict_n, dict_rt])

In [147]:
from sklearn.metrics import mean_squared_error

In [65]:
# проверяем overlap uniform
knn = KNNRegressor(n_neighbors=10, metric=overlap, mode='uniform')
knn.fit(np.array(cat_X_train), np.array(y_train))
prediction = knn.predict(np.array(cat_X_test))

In [66]:
# RMSE for overlap uniform
np.sqrt(mean_squared_error(np.array(y_test), prediction))

229.72851711852473

In [67]:
# проверяем overlap distance
knn = KNNRegressor(n_neighbors=10, metric=overlap, mode='distance')
knn.fit(np.array(cat_X_train), np.array(y_train))
prediction = knn.predict(np.array(cat_X_test))

In [68]:
# RMSE for overlap distance
np.sqrt(mean_squared_error(np.array(y_test), prediction))

231.0524110273355

In [69]:
# проверяем flattened_overlap uniform
knn = KNNRegressor(n_neighbors=10, metric=flattened_overlap, mode='uniform')
knn.fit(np.array(cat_X_train), np.array(y_train))
prediction = knn.predict(np.array(cat_X_test))

In [70]:
# RMSE for flattened_overlap uniform
np.sqrt(mean_squared_error(np.array(y_test), prediction))

220.8353303676556

In [71]:
# проверяем flattened_overlap diatance
knn = KNNRegressor(n_neighbors=10, metric=flattened_overlap, mode='distance')
knn.fit(np.array(cat_X_train), np.array(y_train))
prediction = knn.predict(np.array(cat_X_test))

In [72]:
# RMSE for flattened_overlap distance
np.sqrt(mean_squared_error(np.array(y_test), prediction))

221.2592402239583

In [75]:
# проверяем log_overlap uniform
knn = KNNRegressor(n_neighbors=10, metric=log_overlap, mode='uniform')
knn.fit(np.array(cat_X_train), np.array(y_train))
prediction = knn.predict(np.array(cat_X_test))

In [76]:
# RMSE for log_overlap uniform
np.sqrt(mean_squared_error(np.array(y_test), prediction))

227.3797086881394

In [77]:
# проверяем log_overlap distance
knn = KNNRegressor(n_neighbors=10, metric=log_overlap, mode='distance')
knn.fit(np.array(cat_X_train), np.array(y_train))
prediction = knn.predict(np.array(cat_X_test))

In [78]:
# RMSE for log_overlap distance
np.sqrt(mean_squared_error(np.array(y_test), prediction))

227.69171658343615

Вывод: лучшее качество на RMSE показала функция flattened_overlope uniform. Это связано с тем, что она 

<b>2.3 (1 балл) бонус</b> Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какого удалось достичь уровня качества?

In [86]:
knn = KNNRegressor(n_neighbors=10, metric=flattened_overlap, mode='uniform')
knn.fit(np.array(cat_X_train), np.array(y_train))
distance, index = knn.find_n(np.array(cat_X_test))
weight = np.ones_like(distance)

In [100]:
best_k = -1
best_rmse = -1

y = np.array(y_train)

for k in range(1, 101):
    k_dist = distance[:, :k]
    k_ind = index[:, :k]
    k_weight = weight[:, :k]
    
    ans = np.empty(cat_X_test.shape[0])
    for i, elem in enumerate(k_ind):
        ans[i] = np.sum(y[elem].T * k_weight[i]) / np.sum(k_weight[i])
            
    rmse = np.sqrt(mean_squared_error(np.array(y_test), ans))
    print('rmse = ', rmse, 'k = ', k)
    if best_rmse < 0:
        best_k = k
        best_rmse = rmse
    else:
        if rmse < best_rmse:
            best_rmse = rmse
            best_k = k


rmse =  325.14243437515125 k =  1
rmse =  253.73304782304254 k =  2
rmse =  235.60832624147324 k =  3
rmse =  244.31420723034367 k =  4
rmse =  235.8623027039266 k =  5
rmse =  231.52547201104997 k =  6
rmse =  226.27554964425548 k =  7
rmse =  224.88962376365748 k =  8
rmse =  222.70605572232327 k =  9
rmse =  220.8353303676556 k =  10
rmse =  220.8353303676556 k =  11
rmse =  220.8353303676556 k =  12
rmse =  220.8353303676556 k =  13
rmse =  220.8353303676556 k =  14
rmse =  220.8353303676556 k =  15
rmse =  220.8353303676556 k =  16
rmse =  220.8353303676556 k =  17
rmse =  220.8353303676556 k =  18
rmse =  220.8353303676556 k =  19
rmse =  220.8353303676556 k =  20
rmse =  220.8353303676556 k =  21
rmse =  220.8353303676556 k =  22
rmse =  220.8353303676556 k =  23
rmse =  220.8353303676556 k =  24
rmse =  220.8353303676556 k =  25
rmse =  220.8353303676556 k =  26
rmse =  220.8353303676556 k =  27
rmse =  220.8353303676556 k =  28
rmse =  220.8353303676556 k =  29
rmse =  220.835

In [98]:
# для flattened_overlap
best_k

10

Вывод: оптимальное число соседей 10 (для всех влгоритмов), при дальнейшем росте числа соседей качество не улучшается.

<b>2.4 (2.5 балла)</b> Отойдем ненадолго от задачи регрессии и перейдём к задаче классификации: будем определять, являеться ли квартира дорогой $(target = 1)$ или дешевой $(target = 0)$. Будем считать дорогими квариры, цена которых выше среднего.

In [8]:
data['target'] = (data.price > data.price.mean()).astype(int)

In [9]:
data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,1
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,,0.00,1,365,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,0


Реализуйте счетчики, которые заменят категориальные признаки на вещественные.

А именно, для каждого категориального признака $f_j(x)$ необходимо сделать следующее:
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
\begin{align}
counts_j(c) = \sum_{i=1}^l [f_j(x_i) = c]
\end{align}
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
\begin{align}
successes_j(c) = \sum_{i=1}^l[f_j(x_i) = c][y_i = +1].
\end{align}
3. Сглаженное отношение двух предыдущих величин:
\begin{align}
p_j(c) = \frac{successes_j(c) + a}{counts_j(c) + b},
\end{align}

где $a$ и $b$ - априорные счетчики (например, a = 1, b = 2).

In [10]:
def counts(x):
    # Ваш код здесь
    answer = np.zeros_like(x).astype(int)
    for i, elem in enumerate(x):
        answer[i] = len(np.where(x == elem)[0])
    return answer

def successes(x, y):
    answer = np.zeros_like(x).astype(int)
    for i, elem in enumerate(x):
        answer[i] = len(np.where(y[np.where(x == elem)[0]]  == 1)[0])
    return answer

def p(x, y, a, b):
    suc = successes(x, y)
    count = counts(x)
    return (suc + a) / (count + b)
    

In [74]:
# создаем тесовую и обучающую выборки без фолдов
categoria = ['neighbourhood_group', 'neighbourhood', 'room_type']

X_train_nf, X_test_nf, y_train_nf, y_test_nf = train_test_split(data.drop(columns=['target']), new_data[['target']],
                                                    test_size=0.3, random_state=241)

X_train_nf.drop(columns={'name', 'host_name', 'price', 'last_review', 'id'}, inplace=True)
X_test_nf.drop(columns={'name', 'host_name', 'price', 'last_review', 'id'}, inplace=True)



In [75]:
# преобразуем тестовую выборку
for x in categoria:
    print(x)
    ct = counts(np.array(X_train_nf[x]))
    sc = successes(np.array(X_train_nf[x]), np.array(y_train_nf))
    ans = (sc + 1) / (ct + 2)
    answer = np.zeros(len(X_test_nf[x]))
    for i, elem in enumerate(X_test_nf[x]):
        ind = np.where(np.array(X_train[x]) == elem)[0]
        if len(ind) != 0:
            answer[i] = ans[ind[0]]
        else:
            answer[i] = 0
    X_train_nf[x] = ans
    X_test_nf[x] = answer

neighbourhood_group
neighbourhood
room_type


Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанный по всей обучающей выборке. Реализуйте и такой вариант. Достаточно взять $n = 3$.

In [190]:
def fold_counts(x):
    """
    params:
        x: value on categorical feature for N objects
    returns: vector of length N
    """
    # Ваш код здесь
    ct = np.zeros_like(x)
    
    blocks = len(x) // 3 if len(x) % 3 == 0 else len(x) // 3 + 1
    for q in range(3):
        begin_border = q * blocks
        end_border = min((q + 1) * blocks, len(x))
        sub_array = np.concatenate([x[:begin_border], x[end_border:]])
        cnt = counts(sub_array)
        for i in range(begin_border, end_border):
            ind = np.where(sub_array == x[i])[0]
            if len(ind) == 0:
                ct[i] = 0
            else:
                ct[i] = cnt[ind[0]]
    return ct

def fold_successes(x, y):
    sc = np.zeros_like(x)
    blocks = len(x) // 3 if len(x) % 3 == 0 else len(x) // 3 + 1
    for q in range(3):
        begin_border = q * blocks
        end_border = min((q + 1) * blocks, len(x))
        sub_array = np.concatenate([x[:begin_border], x[end_border:]])
        sub_array_y = np.concatenate([y[:begin_border], y[end_border:]])
        scc = successes(sub_array, sub_array_y)
        for i in range(begin_border, end_border):
            ind = np.where(sub_array_y[np.where(x[i] == sub_array)[0]]  == 1)[0]
            if len(ind) == 0:
                sc[i] = 0
            else:
                sc[i] = scc[ind[0]]
    return sc

def fold_p(x, y, a, b):
    
    sc = np.zeros_like(x)
    ct = np.zeros_like(x)
    
    blocks = len(x) // 3 if len(x) % 3 == 0 else len(x) // 3 + 1
    for q in range(3):
        begin_border = q * blocks
        end_border = min((q + 1) * blocks, len(x))
        train_set = np.arange(begin_border, end_border)
        sub_array = np.concatenate([x[:begin_border], x[end_border:]])
        sub_array_y = np.concatenate([y[:begin_border], y[end_border:]])
        scc = successes(sub_array, sub_array_y)
        cnt = counts(sub_array)
        for i in range(begin_border, end_border):
            ind = np.where(sub_array == x[i])[0]
            #ind_1 = np.where(sub_array_y[ind]  == 1)[0]
            if len(ind) == 0:
                sc[i] = 0
                ct[i] = 0
            else:
                sc[i] = scc[ind[0]]
                ct[i] = cnt[ind[0]]
                
                ct[i] = cnt[ind[0]]
   
    return (sc + a) / (ct + b)

In [177]:
def kfold(n, n_folds):
    if n_folds > n or n_folds < 2:
        raise TypeError
    
    ind_array = np.arange(n)
    if n % n_folds == 0:
        ind_array = np.split(ind_array, n_folds)
    else:
        array = np.split(ind_array[:(n % n_folds) * (n // n_folds + 1)], n % n_folds)
        array = array + np.split(ind_array[(n % n_folds) * (n // n_folds + 1):], n_folds - n % n_folds)
        ind_array = array                             
    answer = []
    for elem in ind_array:
        dop = np.concatenate([np.arange(elem[0]), np.arange(min(elem[::-1][0] + 1, n), n)])
        answer.append((dop, elem))
    return answer

In [178]:
def fold_p(x, y, a, b):
    folds = kfold(len(x), 3)
    answer = np.zeros_like(x)
    for train, test in folds:
        succ = successes(x[train], y[train])
        cnt = counts(x[train])
        
        sс = np.empty(len(test))
        ct = np.empty(len(test))
        
        for enum, item in enumerate(x[test]):
            pos = np.where(x[train] == item)[0]
            if len(pos) == 0:
                suc_test[enum] = 0
                cnt_test[enum] = 0
            else:
                suc_test[enum] = succ[pos[0]]
                cnt_test[enum] = cnt[pos[0]]
                
        answer[test] = (suc_test + a) / (cnt_test + b)
    return answer
        
        

In [191]:
# разбиваем категориальные данные на тестовую и тренировочные выборки чтобы далее сделать преобразование данных по фолдам
# оставляем в данных те же признаки, что ранее
X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(data.drop(columns=['target']),
                                                                        data[['target']],test_size=0.3,
                                                                        random_state=241)

X_train_f.drop(columns={'name', 'host_name', 'price', 'last_review', 'id'}, inplace=True)
X_test_f.drop(columns={'name', 'host_name', 'price', 'last_review', 'id'}, inplace=True)


#for x in categoria:
#    print(x)
#    X_train_f[x] = fold_p(np.array(X_train_f[x]), np.array(y_train_f), 1, 2)


In [192]:
# отдельно преобразуем тестовую выборку
for x in categoria:
    #ct = fold_counts(np.array(X_train_f[x]))
    #sc = fold_successes
    ans = fold_p(np.array(X_train_f[x]), np.array(y_train_f), 1, 2)
    answer = np.zeros(len(X_test_f[x]))
    for i, elem in enumerate(X_test_f[x]):
        ind = np.where(elem == np.array(X_train[x]))[0]
        if len(ind) != 0:
            answer[i] = ans[ind[0]]
        else:
            answer[i] = 0
    X_train_f[x] = ans
    X_test_f[x] = answer

In [199]:
X_train_nf.columns

Index(['neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'minimum_nights', 'number_of_reviews', 'reviews_per_month',
       'calculated_host_listings_count', 'availability_365'],
      dtype='object')

Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [109]:
from sklearn.metrics import roc_auc_score
from sklearn.neighbors import KNeighborsClassifier

X_train_nf.drop(columns=['host_id'], inplace=True)
X_test_nf.drop(columns=['host_id'], inplace=True)

X_train_f.drop(columns=['host_id'], inplace=True)
X_test_f.drop(columns=['host_id'], inplace=True)

In [200]:
X_train_f.drop(columns=['host_id'], inplace=True)
X_test_f.drop(columns=['host_id'], inplace=True)

In [203]:
# без фолдинга
knn = KNeighborsClassifier(n_neighbors=10, metric='euclidean', weights='uniform', algorithm='brute')
knn.fit(np.array(X_train_nf), np.array(y_train_nf))
prediction = knn.predict(np.array(X_test_nf))

C:\Users\ninic\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [204]:
roc_auc_score(np.array(y_test_nf), prediction)

0.60385850821648

In [207]:
# с фолдингом
knn1 = KNeighborsClassifier(n_neighbors=10, metric='euclidean', weights='uniform', algorithm='brute')
knn1.fit(np.array(X_train_f), np.array(y_train_f))
prediction1 = knn1.predict(np.array(X_test_f))

C:\Users\ninic\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [208]:
roc_auc_score(np.array(y_test_f), prediction1)

0.6033059345737839

In [209]:
knn = KNeighborsClassifier(n_neighbors=10, metric='euclidean', weights='uniform', algorithm='brute')
knn.fit(np.array(X_train_f), np.array(y_train_f))
distance, index = knn.find_n(np.array(X_test_f))
weight = np.ones_like(distance)

C:\Users\ninic\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  


AttributeError: 'KNeighborsClassifier' object has no attribute 'find_n'

In [ ]:
best_k = -1
best_rmse = -1

y = np.array(y_train_nf)

for k in range(1, 101):
    k_dist = distance[:, :k]
    k_ind = index[:, :k]
    k_weight = weight[:, :k]
    
    ans = np.empty(X_test_nf.shape[0])
    for i, elem in enumerate(k_ind):
        ans[i] = np.sum(y[elem].T * k_weight[i]) / np.sum(k_weight[i])
            
    err = roc_auc_score(np.array(y_test_f), ans)
    print('roc_auc_sco = ', err, 'k = ', k)
    if best_rmse < 0:
        best_k = k
        best_rmse = rmse
    else:
        if rmse < best_rmse:
            best_rmse = rmse
            best_k = k


<b>2.5 (1 балл)</b> Вернемся к задаче регрессии. Утверждается, что для задачи регрессии можно также сделать преобразование категориальных признаков в действительные числа. Для этого достаточно для каждого значения признака $f_j$ вычислить:
\begin{align}
p_j(c) = g(T_i | f_j(x_i) = c),
\end{align}

где $T_i$ - значения целевой переменной объекта $x_i$. Функция $g$ - среднее (mean) или среднеквадратичное отклонение (std).

Закодируйте категориальные признаки обоими способами и найдите значение RMSE. Используйте евклидову метрику для поиска ближайших соседей. Для какой функции $g$ значение RMSE лучше? Почему?

In [1]:
# Ваш код здесь
def code_cat_mean(x, y):
    answer = np.zeros_like(x)
    for i, elem in enumerate(x):
        ind = np.where(x == elem)[0]
        answer[i] = np.mean(y[ind])
    return answer
def code_cat_std(x, y):
    answer = np.zeros_like(x)
    for i, elem in enumerate(x):
        ind = np.where(x == elem)[0]
        answer[i] = np.std(y[ind])
    return answer

In [141]:
X_train_mn, X_test_mn, y_train_mn, y_test_mn = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

for x in categoria:
    print(x)
    ans = code_cat_mean(np.array(X_train[x]), np.array(y_train))
    answer = np.zeros(len(X_test_mn[x]))
    for i, elem in enumerate(X_test_mn[x]):
        ind = np.where(elem == np.array(X_train[x]))[0]
        if len(ind) != 0:
            answer[i] = ans[ind[0]]
        else:
            answer[i] = 0
    X_train_mn[x] = ans
    X_test_mn[x] = answer
    

neighbourhood_group
neighbourhood
room_type


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,target
39319,30686870,Large bedroom,221940893,Sara,125.588,89.544,40.63628,-73.96970,90.5252,2,12,2019-07-07,3.67,3,220,0
22796,18458323,Sun Drenched Gramercy/East Village!,68274298,Chelsea,197.713,220.979,40.73401,-73.98140,212.716,2,15,2017-12-18,0.58,1,0,1
26190,20886432,Gorgeous single room in Sunnyside near Manhattan,75458625,Lidia,100.009,84.31,40.74722,-73.91860,90.5252,2,20,2019-07-01,0.91,4,365,0
8161,6292334,Cozy UES Studio,7189415,Jennifer,197.713,184.036,40.77357,-73.95193,212.716,1,0,,0.00,1,0,1
32473,25452071,Modern & Cozy Studio In Best East Village Loca...,3142924,Kea,197.713,185.774,40.72714,-73.98215,212.716,2,32,2019-07-06,2.74,1,17,1
35951,28561160,"New York - Private room, perfect location",2570601,Thirza,197.713,208.63,40.78550,-73.97584,90.5252,2,7,2019-03-16,0.80,1,0,0
25954,20728525,Lovely Contemporary Home away from Home,141052003,Nicole,100.009,92.3764,40.68503,-73.77983,212.716,2,45,2019-06-30,2.01,4,153,0
18513,14582793,Nice and comfortable 2/B Apartment,1094833,Meng,125.588,107.823,40.68480,-73.92672,212.716,2,0,,0.00,1,0,0
47680,35867138,New Cozy Studio on the UES with Gym and Pool #...,130270076,Keren,197.713,136.171,40.78672,-73.94880,212.716,30,0,,0.00,2,341,1
30397,23504940,Cozy BK Bungalow,153189324,Mason,125.588,144.875,40.71436,-73.93949,90.5252,2,16,2018-11-12,1.00,2,20,0


In [157]:
X_train_std, X_test_std, y_train_std, y_test_std = train_test_split(data.drop(columns=['price']), data[['price']],
                                                    test_size=0.3, random_state=241)

for x in categoria:
    print(x)
    ans = code_cat_std(np.array(X_train[x]), np.array(y_train))
    answer = np.zeros(len(X_test_std[x]))
    for i, elem in enumerate(X_test_std[x]):
        ind = np.where(elem == np.array(X_train[x]))[0]
        if len(ind) != 0:
            answer[i] = ans[ind[0]]
        else:
            answer[i] = 0
    X_train_std[x] = ans
    X_test_std[x] = answer

neighbourhood_group
neighbourhood
room_type


In [ ]:
X_train_mn.drop(columns=['host_id', 'name', 'id', 'host_name', 'last_review'], inplace=True)
X_test_mn.drop(columns=['host_id', 'name', 'id', 'host_name', 'last_review'], inplace=True)

In [159]:

X_train_std.drop(columns=['host_id', 'name', 'id', 'host_name', 'last_review'], inplace=True)
X_test_std.drop(columns=['host_id', 'name', 'id', 'host_name', 'last_review'], inplace=True)

In [145]:
knn = KNNRegressor(n_neighbors=10, metric='euclidean', mode='distance')
knn.fit(np.array(X_train_mn), np.array(y_train_mn))
prediction = knn.predict(np.array(X_test_mn))

In [148]:
# RMSE for mean
np.sqrt(mean_squared_error(np.array(y_test_mn), prediction))

266.4891244644151

In [160]:
knn = KNNRegressor(n_neighbors=10, metric='euclidean', mode='distance')
knn.fit(np.array(X_train_std), np.array(y_train_std))
prediction1 = knn.predict(np.array(X_test_std))

In [161]:
# RMSE for std
np.sqrt(mean_squared_error(np.array(y_test_std), prediction1))

266.4888209033308

Вывод: лучше работет преобразование признаков через среднеквадратичное, так как 

## Часть 3: Текстовые признаки

<b>3.1 (2 балла)</b> Перейдем от категориальным признаков к текстовым. Рассмотрим 2 способа преобразования текста в действительные числа:
- Мешок слов (Bag of Words)
- TF-IDF

[Здесь](https://scikit-learn.org/stable/modules/feature_extraction.html) вы можете прочитать про их применение в Питоне.

Сравните оба способа на задаче регресси. Какую лучше метрику использовать: евклидову или косинусную меру? Постройте графики зависимости качества решения задачи от способа преобразования, метрики и количества соседей. Мера качества - RMSE.

Объясните полученные результаты.

Перед преобразованием не забудьте уменьшить размер словаря. Например, это можно сделать за счет приведения всех слов к одному регистру и удаления [стопслов](https://en.wikipedia.org/wiki/Stop_words) (артиклей, предлогов, союзов).

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error

In [9]:
vectorizer = CountVectorizer(stop_words='english')
vectorizer1 = CountVectorizer(stop_words='english')

In [10]:
d1 = vectorizer.fit_transform(np.array(data['name'])).toarray()

In [11]:
d2 = vectorizer1.fit_transform(np.array(data['host_name'])).toarray()

In [12]:
dt = np.concatenate([d1, d2], axis=1)

In [13]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(dt, data[['price']],
                                                    test_size=0.3, random_state=241)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
knn.fit(X_train_t, np.array(y_train_t))
prediction = knn.predict(X_test_t)

In [ ]:
np.sqrt(mean_squared_error(np.array(y_test_t), prediction))

In [68]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(vectorizer.fit_transform(data['name']), data[['price']],
                                                    test_size=0.3, random_state=241)

In [69]:
knn = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
knn.fit(X_train_t, np.array(y_train_t))
prediction = knn.predict(X_test_t)

In [70]:
np.sqrt(mean_squared_error(np.array(y_test_t), prediction))

214.77902895349382

In [9]:
# TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english')


In [14]:
from sklearn.neighbors import KNeighborsRegressor

In [15]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(vectorizer.fit_transform(dt), data[['price']],
                                                    test_size=0.3, random_state=241)


In [16]:
knn = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
knn.fit(X_train_t, np.array(y_train_t))
prediction = knn.predict(X_test_t)

In [17]:
np.sqrt(mean_squared_error(np.array(y_test_t), prediction))

206.55723482307377

In [18]:
X_train_t, X_test_t, y_train_t, y_test_t = train_test_split(vectorizer.fit_transform(data['name']), data[['price']],
                                                    test_size=0.3, random_state=241)

In [19]:
knn = KNeighborsRegressor(n_neighbors=10, metric='euclidean')
knn.fit(X_train_t, np.array(y_train_t))
prediction = knn.predict(X_test_t)

In [20]:
np.sqrt(mean_squared_error(np.array(y_test_t), prediction))

214.3534563736316

Вывод: лучше всего сработала обработка текстовых признаков с помощью tf-idf, т.к. она учитывает не только наличие определенных
слов в тексте, но и их "важность"

<b>3.2 (1 балл)</b> Используя все доступные признаки, решите задачу регрессии. Для категориальных и текстовых признаков выберите лучшие преобразования. Повлияло ли добавление количественного признака на метрику качества?

In [7]:
# Ваш код здесь

## Часть 4: Выводы


In [6]:
# Ваши выводы здесь (ノ°∀°)ノ⌒･*:.｡. .｡.:*･゜ﾟ･*☆